<a href="https://colab.research.google.com/github/neurologic/NeurophysiologyModules/blob/main/EMG_MultiChannelEMG_WithVideo_CodingLight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TODO** 
- Make this into a "coding light" version so that it acts more like an app to flexibly interact with analysis of the data... but without requiring looking at all the code. 
- Clean up the kinematics analysis to remove absolute X and Y positions from the analysis (or at least have the option); Include a discussion to help students think about why this would be useful in understanding neural mechanisms or in quantifying and categorizing movements in low-dimensional space. ie. what are the impacts of this choice and why would you want to do it.

<a id="contents"></a>
Whenever you see a link to [*return to table of contents*](#contents), you can return to this table of contents.

# BIG DATA
You have done a lot of work this semester to hone computational skills. This is one of the labs in which the achievement of that learning goal stands out. You can now describe high-dimensional complex data through the lens of contemporary computational tools at the cutting edge of neuroscience research. 

## Laboratory Investigation Steps

1. [Set up our coding environment](#setup)
2. [Set up file path and file name defintions](#filepaths)

**Data from EMG**. *Steps 3-10 are an analysis of the EMG activity*

3. [Import data for EMG](#importemg)
4. [Pick start and stop times for each movement](#definemovements)
5. [Process raw EMG signal into amplitude envelope](#emgprocess)
6. [Calculate pairwise correlations for all EMG signals (amplitude envelope)](#emgcorr)
7. [Dimensionality Reduction on EMG signal](#emgpca)
8. [Calculate total variance explained](#emgpca_var)
9. [Plot PCs that describe the movement](#emgpca_plot)
10. [Plot the EMG activity in PCA space](#emgpcaspace)

**Data from video**. *Steps 11-17 are an analysis of the movement itself*

11. [Import the tracking data of body parts and bones from the video](#movement)
12. [Calculate movement kinematics](#kinematics)
13. [Dimensionality Reduction on the movement](#movementpca)
14. [Calculate total variance explained](#movementpca_var)
15. [Plot PCs that describe the movement](#movementpca_plot)
16. [Look at the movement in 3D PCA space](#movementpcaspace)
17. [Animate the movement in 2D PCA space](#movementpcaspace_animated)

**Write-up (Take home assignment)** 
18. [Take home write up questions and guidelines.](#takehome)

### Throughout the notebook, you will plot raw data and results of analysis.
- You can interact with the plots by zooming in and panning. <br>
- You can make the plot bigger or smaller by dragging its bottom right corner (gray triangle). Note that when it gets smaller the axis labels might disappear.
- You can save the current plot view at any time by hitting the "save" icon - it will save to your Downloads folder. <br>


<a id="setup"></a>
## Part I. Initialize notebook.
[*return to table of contents*](#contents)<br>
As you now know, Python's Jupyter Lab notebooks are a way to combine executable code, code outputs, and text into one connected file. They run using a <b>'kernel'</b>, which is the thing that executes your code. It is what connects the notebook (as you see it) with the part of your computer, or the DataHub computers, that runs code. When you 'run'/'execute' code cells, the kernel does what the code tells it to, for example: importing software packages, importing data, plotting data, calculating correlation between signals, etc. Instructions to 'run' code cells are contained in green boxes within these 'markdown' (text) cells. Make sure to execute the cells in order throughout the notebook without skipping over cells. 

<div class="alert alert-success"><b>Task:</b> Run the cell below to import necessary packages and set up the coding environment.</div>

In [ ]:
# No need to edit anything in this code cell
#################################

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.express as px

import csv
from scipy.signal import hilbert,medfilt,resample
from sklearn.decomposition import PCA
import scipy
import seaborn as sns
import datetime

# https://towardsdatascience.com/how-to-produce-interactive-matplotlib-plots-in-jupyter-environment-1e4329d71651

fs = 100
fs_vid = 30

def get_bodypart_speed(df_vid,d,new_name):
    dd = np.diff(df_vid[d].values)
    fs_ = 1/np.mean(np.diff(df_vid['time']))
    speed = np.concatenate([[0],dd/(1/fs_)])
    filtert = int(0.15*fs_)
    speed = scipy.ndimage.gaussian_filter(speed,filtert)
    df_vid[new_name] = (speed - np.mean(speed))/np.std(speed)
    return df_vid

def get_bodypart_speed_absolute(df_vid,x,y,new_name):
    dx = np.diff(df_vid[x].values)
    dy = np.diff(df_vid[y].values)
    dpos = np.sqrt(dx**2 + dy**2)
    fs_ = 1/np.mean(np.diff(df_vid['time']))
    speed = np.concatenate([[0],dpos/(1/fs_)])
    filtert = int(0.15*fs_)
    speed = scipy.ndimage.gaussian_filter(speed,filtert)
    df_vid[new_name] = (speed - np.mean(speed))/np.std(speed)
    return df_vid

def get_bone_speed(df_vid,bone,new_name):
    dd = np.diff(df_vid[bone].values)
    fs_ = 1/np.mean(np.diff(df_vid['time']))
    speed = np.concatenate([[0],dd/(1/fs_)])
    filtert = int(0.15*fs_)
    speed = scipy.ndimage.gaussian_filter(speed,filtert)
    df_vid[new_name] = (speed - np.mean(speed))/np.std(speed)
    return df_vid

<a id="filepaths"></a>
### Edit the code cell below with the appropriate information, then play/execute the cell
[*return to table of contents*](#contents)<br>
> all file names and paths need to be in quotations, ***if you are on a windows operating system computer, you need an "r" before the first quote of a filepath***
- **folderpath** is the path to your muscle data (.csv file) that has simultaneous recording of two muscles as well as the tracking data (.h5 files) from the video analysis. 
- **emg_file** = the ".csv" data file that has simultaneous recording of two muscles.
- **joints_file** = the "_filtered.h5" file from video tracking analysis
- **bones_file** the "_skeleton.h5" file from video tracking analysis
- **videotimestamps_file** = the ".csv" file that has timestamps from the video camera input during recording
- **channels** = which arduino channels provided your amplified muscle data to bonsai.
- **muscles** = which muscle was recorded on each emg channel.

<div class="alert alert-success"><b>Task:</b> Running the cell below (after entering all of the filepaths and filenames requested) will tell the computer where to look for your data.</div>

In [ ]:
folderpath = '/Volumes/Untitled/BIOL247_Data/20211122_ClassSharedHighDimensional/'

emg_file = "KP_EMG_zip2021-11-22T10_50_38.csv"
joints_file = 'KP_video2021-11-22T10_50_39DLC_resnet101_EMG_Tracking_ZooInitializedNov3shuffle1_58500_filtered.h5'
bones_file = 'KP_video2021-11-22T10_50_39DLC_resnet101_EMG_Tracking_ZooInitializedNov3shuffle1_58500_skeleton.h5'
videotimestamps_file = "KP_video_timestamps2021-11-22T10_50_38.csv"

channels = ['0','1','2','3','4','5'] #channels recorded

muscles = ['tri_L','bi_L','tri_R','bi_R','neck_L','neck_R'] #muscle recorded with each channel


# No need to edit anything below this line
#################################

emg_path = Path(folderpath) / emg_file
joints_path = Path(folderpath) / joints_file
bones_path = Path(folderpath) / bones_file
video_path = Path(folderpath) / videotimestamps_file

print('completed at ' + str(datetime.datetime.now()))

<a id="importemg"></a>
## Part II. Import raw EMG data.
[*return to table of contents*](#contents)<br>
<div class="alert alert-success"><b>Task:</b> Running the cell below actually imports the raw EMG data and formats it in a way that can be plotted.</div>

In [ ]:
#%% import data
df_analog_ = pd.read_csv(emg_path.expanduser().resolve(), sep=',')

# rename headers as input channels
for i,h in enumerate(list(df_analog_.columns)[:-1]):
    df_analog_.rename(columns={h : str(i)},inplace=True)
    
# the last column is timestamps    
df_analog_.rename(columns={list(df_analog_.columns)[-1] : 'time'},inplace=True)

# reformat 'Time' to start at t0 = 0 and be units of seconds
# xtime = (df_analog['Time']-df_analog['Time'].iloc[0])/1000
# reformat 'Time' to be units of seconds
xtime = (df_analog_['time'])/1000
df_analog_['time'] = xtime
x = np.linspace(df_analog_['time'].iloc[0],df_analog_['time'].iloc[-1],
                int((df_analog_['time'].iloc[-1]-df_analog_['time'].iloc[0])*fs))
df_mat = [df_analog_.loc[np.max(df_analog_[df_analog_['time']<=x_].index)].values for x_ in x]
df_analog = pd.DataFrame(df_mat,columns = df_analog_.columns)
df_analog = df_analog.loc[:,channels]
df_analog['time']=x-x[0]

### Plot the raw EMG signal from each muscle
[*return to table of contents*](#contents)<br>
> The amplitude units are in Volts, but the absolute amplitude of the signals are not important for the analyses of this data that we will do for this lab. In the next step, you will be obtaining the "amplitude envelope" of this raw signal, which discards absolute Voltage information anyway. <br>
<br>
> The "baseline" of each channels is offset from each other so that they are not visually overlaid - the baseline voltage of the recording on each channel was actually zero. 

<div class="alert alert-success"><b>Task:</b> Running the cell below plots the raw EMG signal across time from each channel recorded.</div>

In [ ]:
# No need to edit this code cell
################################
plotoffset = 100
hfig,axs = plt.subplots(1)
axs.set_xlabel('seconds')
axs.set_ylabel('amplitude')
axs.set_yticklabels([])
axs.plot(df_analog['time'],df_analog[channels[0]].values,linewidth = 1,label=muscles[0])
axs.plot(df_analog['time'],df_analog[channels[1]].values + plotoffset,linewidth = 1,label=muscles[1])
axs.plot(df_analog['time'],df_analog[channels[2]].values + plotoffset*2,linewidth = 1,label=muscles[2])
axs.plot(df_analog['time'],df_analog[channels[3]].values + plotoffset*3,linewidth = 1,label=muscles[3])
axs.plot(df_analog['time'],df_analog[channels[4]].values + plotoffset*4,linewidth = 1,label=muscles[4])
axs.plot(df_analog['time'],df_analog[channels[5]].values + plotoffset*5,linewidth = 1,label=muscles[5])

plt.legend();

<a id="definemovements"></a>
### Look at the raw data that you just plotted. 
[*return to table of contents*](#contents)<br>
#### Pick a time window to analyze that has decently stereotyped patterned activity for each movement and enter it in the code cell below
- **startA** = the time (seconds) in your recording when nicely stereotyped patterned movement A became clear.
- **stopA** = the time in your recording when nicely stereotyped patterned movement A stopped being clear.
- **startB** = the time in your recording when nicely stereotyped patterned movement B became clear.
- **stopB** = the time in your recording when nicely stereotyped patterned movement B stopped being clear.

You can come back and edit these at any time if you feel like changing this range would clean up your analysis more. All of the rest of the plots in the notebook will reflect these regions that you choose. 

<div class="alert alert-success"><b>Task:</b> Edit the code cell below to define the two regions of your recording used for analysis. Then run the cell.</div>

In [ ]:
startA = 24
stopA = 39

startB = 58
stopB = 72

# No need to edit below this line
################################
print('all set - analysis domain defined')

<a id="emgprocess"></a>
### Process the raw EMG signal into a rectified, smoothed "amplitude envelope"
[*return to table of contents*](#contents)<br>
> This is a type of *signal processing*. You have already encountered this type of signal processing when you analyzed the extracellular cockroach data that you collected earlier this semester. Remind yourself of what this does to the raw signal that you recorded (I explained it thoroughly in the video I made to accompany that analysis). The result here is a processed signal that we will refer to as the *amplitude envelope* of the EMG activity. 

<div class="alert alert-success"><b>Task:</b> Run the cell below to process the raw EMG signal and get its "amplitude envelope". You will also get a plot of the EMG amplitude envelope. You do not need to edit anything in the code cell.</div>

In [ ]:
# No need to edit this code cell
################################

# Use rectfication and gaussian smoothing on EMG to get mean-centered rate
df_rate = pd.DataFrame({})
filtert = int(0.1*fs)
for h in list(df_analog.columns)[:-1]: # rename headers as input channels
    y = df_analog[h] - np.mean(df_analog[h])
    y = np.abs(y) #takes the absolute value of 
    y = scipy.ndimage.gaussian_filter(y,filtert)
    df_rate[h] = y
# df_rate = df_rate.subtract(df_rate.mean())
df_rate =(df_rate - df_rate.mean()) / df_rate.std()
df_rate['time']=df_analog['time']
df_rate = df_rate[((df_rate['time']>startA) & (df_rate['time']<stopA) | (df_rate['time']>startB) & (df_rate['time']<stopB))]

hfig,ax = plt.subplots(1)
ax.set_xlabel('seconds')
ax.set_ylabel('amplitude')
ax.set_yticklabels([])
ax.plot(df_rate['time'],df_rate[channels[0]].values,linewidth = 2,label=muscles[0],alpha = 0.75)
ax.plot(df_rate['time'],df_rate[channels[1]].values,linewidth = 2,label=muscles[1],alpha = 0.75)
ax.plot(df_rate['time'],df_rate[channels[2]].values,linewidth = 2,label=muscles[2],alpha = 0.75)
ax.plot(df_rate['time'],df_rate[channels[3]].values,linewidth = 2,label=muscles[3],alpha = 0.75)
ax.plot(df_rate['time'],df_rate[channels[4]].values,linewidth = 2,label=muscles[4],alpha = 0.75)
ax.plot(df_rate['time'],df_rate[channels[5]].values,linewidth = 2,label=muscles[5],alpha = 0.75)

plt.legend();

<a id="emgcorr"></a>
### Muscle-based description of movement in high-dimensional space.
[*return to table of contents*](#contents)<br>
Think about how you would qualitatively describe the main difference between your movements based on the patterns of muscle acitivity observed for each. How do you describe the movement pattern? What are some of the prominent features that are different? What is similar? 
<div class="alert alert-success"><b>Task:</b> Run the cell below to get a *correlation matrix* table of the mathematical correlation values between EMG signals.</div> 

In [ ]:
# No need to edit below this line
################################
print('correlation matrix movement A:')
inwin = ((df_rate['time']>startA) & (df_rate['time']<stopA))
p_corr = df_rate[inwin].drop('time',axis=1).corr()
# hfig, ax = plt.subplots(1)
# sns.heatmap(p_corr, annot=True)
display(p_corr)

print('correlation matrix movement B:')
inwin = ((df_rate['time']>startB) & (df_rate['time']<stopB))
p_corr = df_rate[inwin].drop('time',axis=1).corr()
# hfig, ax = plt.subplots(1)
# sns.heatmap(p_corr, annot=True)
display(p_corr)

<a id="emgpca"></a>
### Dimensionality Reduction
[*return to table of contents*](#contents)<br>
Again... ***THIS IS A LOT TO KEEP TRACK OF!*** <br>
And it can feel overwhelming to describe accurately without it being confusing with too much detail.
In this section, we will implement "*Dimensionality Reduction*" - a technique commonly used in neuroscience to make sense of "BIG DATA."

<div class="alert alert-success"><b>Task:</b> Run the cell below to perform a "Principal Components Analysis" (PCA) on the ALL of the EMG activity that you recorded (uses the amplitude envelopes). You will get a scatter plot showing how much of the variabce in the data is explained by each principle component.</div>

In [ ]:
# No need to edit this code cell 
################################
df = df_rate.drop('time',axis=1)
df =(df - df.mean()) / df.std()
n_dim = 6
n_components=np.min([n_dim,df.shape[1]]) # if try to take more components than have channels, use amount of channels
print('You quantified your movement into ' + str(len(df.columns)) + ' positional and kinematic variables')
pca = PCA(n_components=n_components)
pca.fit(df)
df_pca = pd.DataFrame(pca.transform(df), columns=['PC%i' % i for i in range(n_components)], index=df.index)
print('Your movement has now been transformed into ' + str(n_components) + ' principle components.')

# No need to edit this code cell
################################
hfig,ax = plt.subplots(1)
ax.scatter(df_pca.columns,pca.explained_variance_ratio_,color='black',s=100)
ax.set_ylabel('Explained Variance')
ax.set_xlabel('Principal Components')
ax.set_ylim(-0.05,1.05)

<a id="emgpca_var"></a>
### Explained variance
[*return to table of contents*](#contents)<br>
Transform the plot above into a 'cumulative variance' plot by using the cell below. For each PC variable listed below, make it equal to the total amount of variance explained at each PC (for example, the total variance explained by PC2 would be the total variance explained by PC0 plus the total variance explained by PC1 plus the total variance explained by PC2). You can get the variance explained by each PC by hovering over the points in the scatter plot above.

<div class="alert alert-success"><b>Task:</b> Edit and run the cell below to make a scatter plot of the cummulative variance explained. A horizontal line will mark the point at which 80% of the total variance is explained by the PCs.</div>

In [ ]:
PC0var = 
PC1var = 
PC2var = 
PC3var = 
PC4var = 
PC5var = 

# No need to edit below this line
################################
hfig,ax = plt.subplots(1)
ax.scatter(df_pca.columns,[PC0var,PC1var,PC2var,PC3var,PC4var,PC5var],color='black',s=100)
ax.hlines(0.8,0,6)
ax.set_ylabel('Total Explained Variance')
ax.set_xlabel('Principal Components')
ax.set_ylim(-0.05,1.05)

<a id="emgpca_plot"></a>
### EMG activity described in PC space.
[*return to table of contents*](#contents)<br>
### PC0 is the first principal component of the movement, PC1 is the second, etc.
> Importantly, these principal components are *independent* of each other. They are *orthogonal* dimensions of the EMG activity.
Edit the following cell so that you can see a plot of all the PCs needed to explain 80% of the variance in EMG activity. 
- find the line that says ```ax.plot(df_rate['time'],df_pca['PC0'],label = 'PC0',alpha = 0.75)```
- If you need to plot more PCs, copy that line (ctrl + C on the keyboard), make a new line, and paste it (ctrl + V on the keyboard).
- change 'PC0' to the next PC you want to plot (for example 'PC1') everywhere in your pasted line that you see it. 
- do this for all PCs needed. <br>

<div class="alert alert-success"><b>Task:</b> Edit the cell below so that you plot the PCs needed to explain at least 80% of the total variance in the movement (based on EMG activity). Run the cell to get a labelled line plot of these PCs overlaid.</div>

In [ ]:
hfig,ax = plt.subplots(1)
ax.set_xlabel('seconds')
ax.set_ylabel('PC units')
ax.set_yticklabels([])

ax.plot(df_rate['time'],df_pca['PC0'],label = 'PC0',alpha = 0.75)


# YOU CAN ADD AS MANY OF THE ax.plt LINES AS YOU NEED TO EXPLAIN 80% of the variance 

plt.legend();

<a id="emgpcaspace"></a>
### EMG space.
[*return to table of contents*](#contents)<br>
Dimensionality reduction techniques, such as principle components analysis (PCA), enable us to represent and quantify high-dimensional things (like neural populations and movements) in a new *space*. <br>
These PCA spaces are in some ways abstract. But in other ways, the abstract space is actually the essence of the neural activity or movement. <br>
What does each movement look like (what is its "shape"?) in the space defined by the PCA dimensions calculated from all movement recorded?

<div class="alert alert-success"><b>Task:</b> Running the cell below plots the first, second, and third principal components against each other in 3D space. Movement A is in purple and Movement B is in green.</div>

In [ ]:

# No need to edit this code cell
################################
fig, ax = plt.subplots(1)
ax = plt.axes(projection='3d')
ax.set_xlabel('Principal Component 0')
ax.set_ylabel('Principal Component 1')
ax.set_zlabel('Principal Component 2')
indA = ((df_rate['time']>startA) & (df_rate['time']<stopA))
indB = ((df_rate['time']>startB) & (df_rate['time']<stopB))
ax.plot3D(df_pca[indA]['PC0'],df_pca[indA]['PC1'],df_pca[indA]['PC2'],color = 'purple',alpha = 0.5,linewidth=2)
ax.plot3D(df_pca[indB]['PC0'],df_pca[indB]['PC1'],df_pca[indB]['PC2'],color = 'green',alpha = 0.5,linewidth=2);


<a id="movement"></a>
## Part III. Movement analysis
[*return to table of contents*](#contents)<br>
Now let's examine how motor system activity relates to movement kinematics.
### Edit the code cell below with the appropriate information, then play/execute the cell
- **bodyparts** is the list of body parts that are relevant to your analysis of the movement/muscle activity. Each body part must be in quotes. Body parts separated by commas.
- **bones** = is the list of bones that are relevant to your analysis of the movement/muscle activity. The names of your bones are determined by how you constructed your skeleton. Each bone must be in quotes. Bones separated by commas.
<br>

#### Possible body parts: (1 = right, 2 = left)
- wrist1(or 2), elbow1(or 2), shoulder1(or 2), hip1(or 2), knee1(or 2), ankle1(or 2), chin, forehead.

#### Possible bones: 
- refer to the skeleton that you created for your video tracking in Collaboratory notebook. 

<div class="alert alert-success"><b>Task:</b> Running the cell below will import and collect tracking data for the body parts and bones that you specify.</div>

In [ ]:
bodyparts = ['wrist1','elbow1','elbow2',...]
bones = ['wrist1_elbow1','elbow1_shoulder1',...]



# No need to edit below this line
################################

df_vid = pd.read_hdf(joints_path) # import data from body parts _filtered.h5
df_vid = df_vid[df_vid.columns[0][0]]
df_vid.columns = df_vid.T.index.map('_'.join)
coords = ['_x','_y']
# filter by coordinates or liklihood
coords = [any(keep in ele for keep in coords) for ele in list(df_vid.columns)]
# filter by bodyparts
bodyparts_ind = [any(keep in ele for keep in bodyparts) for ele in list(df_vid.columns)]
df_vid_bodyparts = df_vid.loc[:,(np.asarray(coords) & np.asarray(bodyparts_ind))]

df_vid = pd.read_hdf(bones_path) # import data from bones _skeleton.h5
df_vid.columns = df_vid.T.index.map('_'.join)
coords = ['orientation']
# filter by coordinates or liklihood
coords = [any(keep in ele for keep in coords) for ele in list(df_vid.columns)]
# filter by bodyparts
bones_ind = [any(keep in ele for keep in bones) for ele in list(df_vid.columns)]
df_vid_bones = df_vid.loc[:,(np.asarray(coords) & np.asarray(bones_ind))]

df_vid = df_vid_bones.join(df_vid_bodyparts) # combine bones and bodyparts
df_vid =(df_vid - df_vid.mean()) / df_vid.std() # mean subtract and normalize by std
df_vid['time'] = df_vid.index.values / fs_vid # create time column
df_vid = df_vid[((df_vid['time']>startA) & (df_vid['time']<stopA)) | ((df_vid['time']>startB) & (df_vid['time']<stopB))] # select start:stop section

print('body parts and bones that you can now analyze for kinematics:')
print(list(df_vid.keys()))
print('')
print('sample of table with positions of body parts and orientations of bones')
display(df_vid.head())



<a id="kinematics"></a>
### Calculate some basic movement kinematics - speed
[*return to table of contents*](#contents)<br>

<div class="alert alert-success"><b>Task:</b> Running the cell below gets the speed of each body part in _x and _y  and the speed of each bone (in degrees). You only need to run this cell once. It automatically calculates the speed of every body part and bone that you imported (defined by bodyparts and bones lists in previous cell). You will be able to see the new table with all of these columns added (you actually won't be able to see all the columns because the table is too wide to show here).</div>

In [ ]:
# No need to edit this code cell
################################
print('movement parameters quantified:')
for b in bodyparts:
    print(b)
    d = b + '_x'
    new_name = b + '_xspeed'
    print(new_name)
    df_vid = get_bodypart_speed(df_vid,d,new_name)
    
for b in bodyparts:
    d = b + '_y'
    new_name = b + '_yspeed'
    print(new_name)
    df_vid = get_bodypart_speed(df_vid,d,new_name)
    
for b in bones:
    b = b + '_orientation'
    print(b)
    new_name = b + '_speed'
    print(new_name)
    df_vid = get_bone_speed(df_vid,b,new_name)

# No need to edit below this line
################################
display(df_vid.drop('time',axis=1).head())
print('body parts, bones, and kinematics that you can now examine and compare to muscle activity:')
print(list(df_vid.drop('time',axis=1).keys()))

<a id="movementpca"></a>
### *Dimensionality Reduction* of movement.
[*return to table of contents*](#contents)<br>
> We won't even bother plotting individual movement kinematics this time... why? Do you see how many variables there are to plot!? And we only quantified speed! We could also look at acceleration, etc. 

<div class="alert alert-success"><b>Task:</b> Run the cell below to perform a "Principal Components Analysis" (PCA) on the ALL of the tracking and kinematics data that you have used to quantify the movement. You will get a report printed out of how many variables the movement was originally described by and how many PCs the movement was reduced to (I have set this to 10). You will get a scatter plot showing how much of the variance in the data is explained by each principle component.</div>

In [ ]:
# No need to edit this code cell the first time you run in
################################
df = df_vid.drop('time',axis=1)
df =(df - df.mean()) / df.std()
n_dim = 10
n_components=np.min([n_dim,df.shape[1]]) # if try to take more components than have channels, use amount of channels
print('You quantified your movement into ' + str(len(df.columns)) + ' positional and kinematic variables')
pca = PCA(n_components=n_components)
pca.fit(df)
df_pca = pd.DataFrame(pca.transform(df), columns=['PC%i' % i for i in range(n_components)], index=df.index)
print('Your movement has now been transformed into ' + str(n_components) + ' principle components.')


# No need to edit this code cell
################################
hfig,ax = plt.subplots(1)
ax.scatter(df_pca.columns,pca.explained_variance_ratio_,color='black',s=100)
ax.set_ylabel('Explained Variance')
ax.set_xlabel('Principal Components')
ax.set_ylim(-0.05,1.05)

<a id="movementpca_var"></a>
### Explained variance
[*return to table of contents*](#contents)<br>
Transform the plot above into a 'cumulative variance' plot by using the cell below. For each PC variable listed below, make it equal to the total amount of variance explained at each PC (for example, the total variance explained by PC2 would be the total variance explained by PC0 plus the total variance explained by PC1 plus the total variance explained by PC2). 

<div class="alert alert-success"><b>Task:</b> Edit and run the cell below to make a scatter plot of the cummulative variance explained. The horizontal line marks the point at which 80% of the total variance is explained by the PCs.</div>

In [ ]:
PC0var = 
PC1var = 
PC2var = 
PC3var = 
PC4var = 
PC5var = 
PC6var = 
PC7var = 
PC8var = 
PC9var = 

# No need to edit below this line
################################
hfig,ax = plt.subplots(1)
cummsum = [PC0var,PC1var,PC2var,PC3var,PC4var,PC5var,PC6var,PC7var,PC8var,PC9var]
ax.scatter(df_pca.columns[0:len(cummsum)],cummsum,color='black',s=100)
ax.hlines(0.8,0,len(cummsum))
ax.set_ylabel('Total Explained Variance')
ax.set_xlabel('Principal Components')
ax.set_ylim(-0.05,1.05)

<a id="movementpca_plot"></a>
### Movement activity described in PC space.
[*return to table of contents*](#contents)<br>
PC0 is the first principal component of the movement, PC1 is the second, etc.
> Importantly, these principal components are *independent* of each other. They are *orthogonal* dimensions of the movement.
Edit the following cell so that you can see a plot of all the PCs needed to explain 80% of the variance in EMG activity. 
- find the line that says ```ax.plot(df_vid['time'],df_pca['PC0'],label = 'PC0',alpha = 0.75)```
- If you need to plot more PCs, copy that line (ctrl + C on the keyboard), make a new line, and paste it (ctrl + V on the keyboard).
- change 'PC0' to the next PC you want to plot (for example 'PC1') everywhere in your pasted line that you see it. 
- do this for all PCs needed. <br>

<div class="alert alert-success"><b>Task:</b> Edit the cell below so that you plot the PCs needed to explain at least 80% of the total variance in the movement (based on EMG activity). Run the cell to get a labelled line plot of these PCs overlaid.</div>

In [ ]:
hfig,ax = plt.subplots(1)
ax.set_xlabel('seconds')
ax.set_ylabel('PC units')
ax.set_yticklabels([])

ax.plot(df_vid['time'],df_pca['PC0'],label = 'PC0',alpha = 0.75)


# YOU CAN ADD AS MANY OF THE ax.plt LINES AS YOU NEED TO EXPLAIN 80% of the variance 

plt.legend();

<a id="movementpcaspace"></a>
### Movement PCA space
[*return to table of contents*](#contents)<br>
Dimensionality reduction techniques, such as principle components analysis (PCA), enable us to represent and quantify high-dimensional things (like neural populations and movements) in a new *space*. 
These PCA spaces are in some ways abstract. But in other ways, the abstract space is actually the essence of the neural activity or movement. 
What does each movement look like (what is its "shape"?) in the space defined by the PCA dimensions calculated from all movement recorded?

<div class="alert alert-success"><b>Task:</b> Running the cell below plots the first, second, and third principal components against each other in 3D space. Movement A is in purple and Movement B is in green.</div>

In [ ]:

# No need to edit this code cell
################################
fig, ax = plt.subplots(1)
ax = plt.axes(projection='3d')
ax.set_xlabel('Principal Component 0')
ax.set_ylabel('Principal Component 1')
ax.set_zlabel('Principal Component 2')
indA = ((df_vid['time']>(startA+0.5)) & (df_vid['time']<(stopA-0.5)))
indB = ((df_vid['time']>(startB+0.5)) & (df_vid['time']<(stopB-0.5)))
ax.plot3D(df_pca[indA]['PC0'],df_pca[indA]['PC1'],df_pca[indA]['PC2'],color = 'purple',alpha = 0.5,linewidth=2)
ax.plot3D(df_pca[indB]['PC0'],df_pca[indB]['PC1'],df_pca[indB]['PC2'],color = 'green',alpha = 0.5,linewidth=2);


<a id="movementpcaspace_animated"></a>
### How does the movement evolve over time? (ie. where is time in the plot?)
[*return to table of contents*](#contents)<br>
<div class="alert alert-success"><b>Task:</b> Running the cell below animates the movement over time through this "latent" space - the space defined by the principal components.</div>

In [ ]:
# No need to edit this code cell
################################

fig, ax = plt.subplots(1)
ax.plot(df_pca['PC0'],df_pca['PC1'],color = 'black',alpha = 0.5,linewidth=2)
t, = ax.plot(df_pca['PC0'].values[0], df_pca['PC1'].values[0],color = 'red',marker='.',markersize = 20)

def animate(i):
    ax.plot(df_pca['PC0'].values[i], df_pca['PC1'].values[i],color = 'red',marker='.',markersize = 15,zorder=3,alpha = 0.25)
    
ani = animation.FuncAnimation(
    fig, animate, frames = len(df_pca),interval=10, repeat = False)
plt.show()

<a id="takehome"></a>
# Take-home assignmnent
[*return to table of contents*](#contents)<br>

### Figure 1
Use the plot of the EMG amplitude envelop (with the processed signal from each muscle overlaid). Create a figure containing 2 full periods/cycles of each rhythmic patterned movement (one figure for each movement).
Visually indicate the start of one cycle and the stop of that cycle. With words, describe: which muscle cells were active (spiking) in phase and which were out of phase with each other?, what was the pattern of co-activation over time? It might not make sense to do this for every little deflection you see in the amplitude envelope - so decide what gives you a balance between a clear and an accurate description. Make a conclusive qualitative statement about the main difference between the two movements based on the pattern of EMG activity (this is a statement that you would want to quantify somehow in order to study more about the difference between these movements).

### Figure 2
Re-label the correlation matrix rows and columns with your muscles. What are some of the differences between the two movements in terms of correlation patterns? Again, instead of focusing on listing every comparison, use the quantification to make concise and useful qualitative statements about how the pattern of muscle activation differs. 

### Figure 3
Using the appropriate code cells in the notebook... 
Panel A) EMG PCA - Calculate the cummulative variance explained and make a scatter plot of that result. 
Panel B) Make an overlay plot of the PCs that you need to explain at least 80% of the variance. Similar to FIgure 1, describe the pattern of EMG activity using the pattern of PC activation.
Panel C and D) Same as A and B, but for the movement kinematics instead of the EMG signals. 

### Figure 4
Panels A, B, C, D, E, F: 3 columns, 2 rows of panels.
Columns: Movement A, Movement B, Movement A vs B
Rows: EMG, Video
Rotate the 3D plots of the EMG/video data in PC space (PC0 vs PC1 vs PC2) to get 2D figures (using the "save as" button) for each panel. For the Movement A column, rotate the PC space until you find a view that shows movement A best (the plan along which movement A varies the most in 2 dimensions). Movement B column, same as A but for movement B. Movement A vs B column, rotate the PC space until you find a plane that shows the most separation between the two movements.
Relate the Movement A vs B results back to what you showed in Figure 3 B&D. In other words, how does the variance in each PC in Figure 3B vs D relate to the plane in 3D PC space on which the two movements are the most seperable/different?
